In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


In [13]:
# Any results you write to the current directory are saved as output.
import time
import lightgbm as lgb
from sklearn.model_selection import KFold
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
def add_agg(merged_df, gr_cols, new_col_name, incr_yr):
    agg1 = train_df.groupby(gr_cols)['target'].agg('mean').reset_index()
    cols2 = gr_cols.copy()
    cols2.append(new_col_name)
    agg1.columns = cols2
    if incr_yr:
        agg1['year']+=1
    merged_df = pd.merge(merged_df, agg1, how='left', left_on=gr_cols, right_on=gr_cols)
    return merged_df

In [15]:
# read raw data
start_time = time.time()
print('  Loading data...')
input_dir  = os.path.join(os.pardir, 'input')
train_df   = pd.read_csv(os.path.join(input_dir, 'train.csv'), nrows=None)
test_df    = pd.read_csv(os.path.join(input_dir, 'test.csv'), nrows=None)
print('    Time elapsed %.0f sec'%(time.time()-start_time))

  Loading data...
    Time elapsed 0 sec


In [16]:
# Merge test/train datasets into a single one and separate unneeded columns
merged_df = pd.concat([train_df, test_df], sort=False)

In [17]:
merged_df.head()

,date,store,item,sales,id
0,2013-01-01,1,1,13.0,NaN
1,2013-01-02,1,1,11.0,NaN
2,2013-01-03,1,1,14.0,NaN
3,2013-01-04,1,1,13.0,NaN
4,2013-01-05,1,1,10.0,NaN


In [19]:
df1 = pd.DataFrame({'Col1': [10, 10, 15, 15, 15, 45],
                   'Col2': [13, 23, 18, 33, 14, 48]})
df1

,Col1,Col2
0,10,13
1,10,23
2,15,18
3,15,33
4,15,14
5,45,48


In [21]:
df2 = pd.DataFrame({'Col2': [14, 23, 18, 33, 14, 49],
                      'Col3': [17, 27, 22, 37, 28, 52]})
df2

,Col2,Col3
0,14,17
1,23,27
2,18,22
3,33,37
4,14,28
5,49,52


In [23]:
res = pd.concat( [df1, df2] , sort=False)
res

,Col1,Col2,Col3
0,10.0,13,NaN
1,10.0,23,NaN
2,15.0,18,NaN
3,15.0,33,NaN
4,15.0,14,NaN
5,45.0,48,NaN
0,NaN,14,17.0
1,NaN,23,27.0
2,NaN,18,22.0
3,NaN,33,37.0


In [24]:
# add columns: date related
merged_df['date']  = pd.to_datetime(merged_df['date'],infer_datetime_format=True)
merged_df['year']  = merged_df['date'].dt.year
merged_df['month']  = merged_df['date'].dt.month
merged_df['day']   = merged_df['date'].dt.dayofweek
merged_df.drop('date', axis=1, inplace=True)

In [25]:
merged_df.head()

,store,item,sales,id,year,month,day
0,1,1,13.0,NaN,2013,1,1
1,1,1,11.0,NaN,2013,1,2
2,1,1,14.0,NaN,2013,1,3
3,1,1,13.0,NaN,2013,1,4
4,1,1,10.0,NaN,2013,1,5


In [26]:
# add grouped columns
train_df=pd.DataFrame(merged_df[merged_df.sales.notna()].values)
train_df.columns=merged_df.columns
train_df['target']=train_df['sales'] # rename

In [27]:
train_df.head()

,store,item,sales,id,year,month,day,target
0,1.0,1.0,13.0,NaN,2013.0,1.0,1.0,13.0
1,1.0,1.0,11.0,NaN,2013.0,1.0,2.0,11.0
2,1.0,1.0,14.0,NaN,2013.0,1.0,3.0,14.0
3,1.0,1.0,13.0,NaN,2013.0,1.0,4.0,13.0
4,1.0,1.0,10.0,NaN,2013.0,1.0,5.0,10.0


In [8]:
# scale for item+store+year(prev). Need to scale predictions back up!
merged_df = add_agg(merged_df,['item','store','year'], 'tsy', 1)
merged_df['sales']/=merged_df['tsy']
merged_df = merged_df[merged_df.year>2013]
tsy=merged_df.pop('tsy')

# pop sales and ID
ID=merged_df[merged_df.id.notna()]['id']
target=merged_df[merged_df.sales.notna()]['sales']
merged_df.drop(['id','sales'], axis=1, inplace=True)
len_train=target.shape[0]

# use lightgbm for regression
print('    Time elapsed %.0f sec'%(time.time()-start_time))

    Time elapsed 50 sec


In [10]:
# specify your configurations as a dict
params = {
    'nthread': 10,
    'max_depth': 8,
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression_l1',
    'metric': 'mape', # this is abs(a-e)/max(1,a)
    'num_leaves': 31,
    'learning_rate': 0.25,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l1': 0.06,
    'lambda_l2': 0.1,
    'verbose': -1
}

# do the training
num_folds = 5
test_x = merged_df[len_train:].values
all_x = merged_df[:len_train].values
all_y = target.values
oof_preds = np.zeros([all_y.shape[0]])
sub_preds = np.zeros([test_x.shape[0]])
feature_importance_df = pd.DataFrame()
folds = KFold(n_splits=num_folds, shuffle=True, random_state=345665)
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(all_x)):
    train_x, train_y = all_x[train_idx], all_y[train_idx]
    valid_x, valid_y = all_x[valid_idx], all_y[valid_idx]
    lgb_train = lgb.Dataset(train_x,train_y)
    lgb_valid = lgb.Dataset(valid_x,valid_y)
    
     # train
    gbm = lgb.train(params, lgb_train, 1000, 
        valid_sets=[lgb_train, lgb_valid],
        early_stopping_rounds=100, verbose_eval=100)
    oof_preds[valid_idx] = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
    sub_preds[:] += gbm.predict(test_x, num_iteration=gbm.best_iteration) / folds.n_splits
    valid_idx += 1
    importance_df = pd.DataFrame()
    importance_df['feature'] = merged_df.columns
    importance_df['importance'] = gbm.feature_importance()
    importance_df['fold'] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, importance_df], axis=0)
e = 2 * abs(all_y - oof_preds) / ( abs(all_y)+abs(oof_preds) )
e = e.mean()
print('Full validation score %.4f' %e)

Training until validation scores don't improve for 100 rounds.
[100]	training's mape: 0.109105	valid_1's mape: 0.109617
Early stopping, best iteration is:
[18]	training's mape: 0.109478	valid_1's mape: 0.109514
Training until validation scores don't improve for 100 rounds.
[100]	training's mape: 0.109119	valid_1's mape: 0.109708
Early stopping, best iteration is:
[60]	training's mape: 0.109282	valid_1's mape: 0.109658
Training until validation scores don't improve for 100 rounds.
[100]	training's mape: 0.10914	valid_1's mape: 0.109653
Early stopping, best iteration is:
[21]	training's mape: 0.109471	valid_1's mape: 0.109601
Training until validation scores don't improve for 100 rounds.
[100]	training's mape: 0.109189	valid_1's mape: 0.109466
Early stopping, best iteration is:
[19]	training's mape: 0.109547	valid_1's mape: 0.109404
Training until validation scores don't improve for 100 rounds.
[100]	training's mape: 0.109124	valid_1's mape: 0.109645
Early stopping, best iteration is:
[2

In [11]:
# Write submission file
pred = (sub_preds * tsy[len_train:] ).astype(np.float32)
out_df = pd.DataFrame({'id': ID.astype(np.int32), 'sales': pred})
out_df.to_csv('submission.csv', index=False)
print('    Time elapsed %.0f sec'%(time.time()-start_time))



    Time elapsed 152 sec
